In [1]:
from pyhealth.datasets import MIMIC3Dataset
from pyhealth.datasets import split_by_patient, get_dataloader
from pyhealth.models import Transformer, RNN, RETAIN
from pyhealth.tasks import mortality_prediction_mimic3_fn, drug_recommendation_mimic3_fn
from pyhealth.trainer import Trainer

dataset = MIMIC3Dataset(
    root='/data/physionet.org/files/mimiciii/1.4/',
    tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
    code_mapping={
        # "ICD9CM": "CCSCM", 
        # "ICD9PROC": "CCSPROC",
        "NDC": ("ATC", {"target_kwargs": {"level": 3}})
        },
    dev=False,
    refresh_cache=True
)


/home/pj20/miniconda3/envs/txgnn_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
finish basic patient information parsing : 5.599072456359863s
finish parsing DIAGNOSES_ICD : 9.125769138336182s
finish parsing PROCEDURES_ICD : 6.70231294631958s
finish parsing PRESCRIPTIONS : 58.10236859321594s


Mapping codes: 100%|██████████| 46520/46520 [01:21<00:00, 570.40it/s] 


In [2]:
mimic3_ds = dataset.set_task(drug_recommendation_mimic3_fn)

train_dataset, val_dataset, test_dataset = split_by_patient(
    mimic3_ds, [0.8, 0.1, 0.1]
)
train_dataloader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

Generating samples for drug_recommendation_mimic3_fn: 100%|██████████| 46520/46520 [00:02<00:00, 21575.02it/s]


In [4]:
model_w_pre = RNN(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="drugs",
    mode="multilabel",
    pretrained_emb="LM/clinicalbert",
    embedding_dim=128,
)

Loading pretrained embedding for conditions...
Loading pretrained embedding for procedures...


In [5]:
# STEP 4: define trainer
trainer = Trainer(model=model_w_pre)
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=30,
    monitor="pr_auc_samples",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(4493, 768, padding_idx=0)
    (procedures): Embedding(1414, 768, padding_idx=0)
  )
  (linear_layers): ModuleDict(
    (conditions): Linear(in_features=768, out_features=128, bias=True)
    (procedures): Linear(in_features=768, out_features=128, bias=True)
  )
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
    (procedures): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=256, out_features=193, bias=True)
)
Metrics: None
Device: cuda

Training:
Batch size: 32
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7f25c3cbb4c0>
Monitor: pr_auc_samples
Monitor criterion: max
Epochs: 30



Epoch 0 / 30: 100%|██████████| 354/354 [00:03<00:00, 106.94it/s]

--- Train epoch-0, step-354 ---
loss: 0.2926



Evaluation: 100%|██████████| 45/45 [00:00<00:00, 255.78it/s]


--- Eval epoch-0, step-354 ---
pr_auc_samples: 0.6669
loss: 0.2808
New best pr_auc_samples score (0.6669) at epoch-0, step-354



...

--- Train epoch-29, step-10620 ---
loss: 0.2696



Evaluation: 100%|██████████| 45/45 [00:00<00:00, 248.80it/s]


--- Eval epoch-29, step-10620 ---
pr_auc_samples: 0.6822
loss: 0.2633
Loaded best model


Evaluation: 100%|██████████| 44/44 [00:00<00:00, 257.54it/s]


{'pr_auc_samples': 0.6888944454447027, 'loss': 0.2677140161395073}


In [5]:
model_no_pre = RNN(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="drugs",
    mode="multilabel",
    embedding_dim=128,
)

In [6]:
# STEP 4: define trainer
trainer = Trainer(model=model_no_pre)
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=10,
    monitor="pr_auc_samples",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

RNN(
  (rand_init_embedding): ModuleDict()
  (pretrained_embedding): ModuleDict()
  (embeddings): ModuleDict(
    (conditions): Embedding(4493, 128, padding_idx=0)
    (procedures): Embedding(1414, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
    (procedures): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=256, out_features=193, bias=True)
)
Metrics: None
Device: cuda

Training:
Batch size: 32
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7ff51064a8b0>
Monitor: pr_auc_samples
Monitor criterion: max
Epochs: 10



Epoch 0 / 10: 100%|██████████| 354/354 [00:02<00:00, 125.89it/s]

--- Train epoch-0, step-354 ---
loss: 0.3037



Evaluation: 100%|██████████| 44/44 [00:00<00:00, 244.18it/s]


--- Eval epoch-0, step-354 ---
pr_auc_samples: 0.7173
loss: 0.2600
New best pr_auc_samples score (0.7173) at epoch-0, step-354



...

--- Train epoch-9, step-3540 ---
loss: 0.2262



Evaluation: 100%|██████████| 44/44 [00:00<00:00, 230.73it/s]


--- Eval epoch-9, step-3540 ---
pr_auc_samples: 0.7431
loss: 0.2434
Loaded best model


Evaluation: 100%|██████████| 45/45 [00:00<00:00, 243.22it/s]


{'pr_auc_samples': 0.7394630422926343, 'loss': 0.2355364070998298}
